In [1]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [2]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [3]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

In [4]:
newExChangeInfo,infopayload = AppExchangeInfo()

In [5]:
ffi.string(newExChangeInfo.vendor)

b'nobody'

In [6]:
ffi.string(newExChangeInfo.name)

b'noName'

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [7]:
def PermissionSet(read=False,insert=False,update=False,delete=False,manage_permissions=False):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

In [8]:
newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [9]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

In [10]:
newPermissions,permission_addon=ContainerPermissions(b'myName')

In [11]:
newPermissions.access.insert

False

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [12]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

In [13]:
myAuth,addData=AuthReq(newPermissions,1,1)

In [14]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [15]:
lib.app_set_additional_search_path(ffi.new('char[]',b'/home/riddim/maidsafe/pySafe/compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


In [16]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

In [17]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
<cdata 'FfiResult *' 0x7ffda2c73bc8>
0
<cdata 'char *' NULL>


In [18]:
myAuth.containers.access.insert=True

In [19]:
myAuth.containers.access.delete=True

In [20]:
encodedAuth=None

In [21]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    global encodedAuth
    print(userData)
    print(result.error_code)
    print(noClueWhatNumber)
    print(ffi.string(resultString))
    encodedAuth = ffi.string(resultString)

In [22]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
2147592359
b'AAAAAKeoAYAAAAAABAAAAAAAAABub0lkAQcAAAAAAAAAbm9TY29wZQYAAAAAAAAAbm9OYW1lBgAAAAAAAABub2JvZHkBAQAAAAAAAAAGAAAAAAAAAG15TmFtZQIAAAAAAAAAAQAAAAMAAAA'


In [23]:
from ctypes import *

In [24]:
sysUri=cdll.LoadLibrary('../compiled_binaries/libsystem_uri.so')

In [ ]:
sysUri.open(encodedAuth)

-1

here the kernel always dies ... i'm doing something very wrong! ...

In [ ]:
sysUri.install(encodedAuth)